# Strategy

1. We used opensource Llama 2 70B model (https://huggingface.co/spaces/ysharma/Explore_llamav2_with_TGI) to generate additional datapoints for classes that had less datapoints ("More Likely" and "Necessarily True").
2. We gave the Llama 2 model some examples of our sentences and the structure we want, and asked it to make similar data points.
3. These data points were further verified by us manually to look for any outliers.

In [64]:
!ls ../../datasets/adept/generated-data/

3.csv  4.csv  artificial_train_combined.csv  generated-label-3.json


In [42]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import json

### Compiling Data generated by Sarthak

In [43]:
adept_data_path = "../../datasets/adept/train-dev-test-split"
split = "train"
artificial_adept_data_path = "../../datasets/adept/generated-data/"

In [44]:
train_data = json.load(open('{}/{}.json'.format(adept_data_path, split), 'r'))

In [45]:
df_train = pd.DataFrame(train_data)
df_train.head()

,sentence1,sentence2,modifier,noun,label,idx
0,The effect of sleeping is rejuvenation.,The effect of additional sleeping is rejuvenat...,additional,sleeping,3,13484
1,A toothbrush is for fresh breath.,A regular toothbrush is for fresh breath.,regular,toothbrush,2,2620
2,A scene is painted.,A negative scene is painted.,negative,scene,2,3324
3,A bone breaks a tooth.,An alleged bone breaks a tooth.,alleged,bone,2,10610
4,A trip causes a happening.,A fabulous trip causes a happening.,fabulous,trip,2,14917


In [46]:
df_4 = pd.read_csv("{}/{}.csv".format(artificial_adept_data_path, "4"))
df_4["label"] = 4
df_3 = pd.read_csv("{}/{}.csv".format(artificial_adept_data_path, "3"))
df_3["label"] = 3

In [47]:
df_4.columns

Index(['Noun', 'Adjective Modifier', 'Verb', 'Sentence', 'label'], dtype='object')

In [48]:
del df_4['Verb']
del df_3['Verb']

In [49]:
df_4.rename({"Noun": "noun", "Adjective Modifier": "modifier", "Sentence": "sentence2"}, axis=1, inplace=True)
df_3.rename({"Noun": "noun", "Adjective Modifier": "modifier", "Sentence": "sentence2"}, axis=1, inplace=True)

In [50]:
df_artificial = pd.concat([df_4, df_3])
df_artificial.head(2)

,noun,modifier,sentence2,label
0,Dog,Happy,A happy dog wags its tail.,4
1,Building,Tall,A tall building casts a long shadow.,4


In [51]:
df_artificial.shape

(177, 4)

### Compiling data generated by Tonmoy

In [52]:
artificial_data = json.load(open('{}/{}.json'.format(artificial_adept_data_path, "generated-label-3"), 'r'))

In [53]:
df_artificial2 = pd.DataFrame(artificial_data)
df_artificial2.head()

,sentence1,sentence2,modifier,noun,label,idx
0,Retrievers wag friendly tails.,Golden retrievers wag friendly tails.,Golden,retrievers,3,0
1,Bacon fills kitchens with smoke.,Sizzling bacon fills kitchens with smoke.,Sizzling,bacon,3,1
2,Coffee aromas fill rooms.,Brewed coffee aromas fill rooms.,Brewed,coffee,3,2
3,Sunshine warms skin nicely.,Bright sunshine warms skin nicely.,Bright,sunshine,3,3
4,Kittens purr contentedly.,Fluffy kittens purr contentedly.,Fluffy,kittens,3,4


In [54]:
df_artificial2.shape

(117, 6)

### Combining training and artificial datasets

In [59]:
df_combined = pd.concat([df_artificial, df_artificial2, df_train], ignore_index=True)
df_combined.head(2)

,noun,modifier,sentence2,label,sentence1,idx
0,Dog,Happy,A happy dog wags its tail.,4,NaN,NaN
1,Building,Tall,A tall building casts a long shadow.,4,NaN,NaN


In [60]:
df_combined.tail(2)

,noun,modifier,sentence2,label,sentence1,idx
13184,farmer,ignorant,An ignorant farmer shakes a tree.,2,A farmer shakes a tree.,8846.0
13185,closet,essential,An essential closet is used for keeping your c...,2,A closet is used for keeping your clothes in.,13281.0


In [63]:
df_combined.shape

(13186, 6)

In [62]:
df_combined.to_csv('{}/artificial_train_combined.csv'.format(artificial_adept_data_path), index=False)